In [1]:
import re
import urllib.request
import urllib.error
import json
import random

from os import listdir
import csv
import time
import pypinyin

import pandas as pd

In [26]:
"""
设置需要爬取的地区，使用该地区的拼音
设置需要爬取的年、月
"""
path='/Users/rumeng/Downloads/qingzang/trainSet/test/'
filelist=listdir(path)

# 不带声调的(style=pypinyin.NORMAL)
def pinyin(word):
    s = ''
    for i in pypinyin.pinyin(word, style=pypinyin.NORMAL):
        s += ''.join(i)
    return s

cities=[]
# # cities = ['shenzhen','shanghai','guangzhou','beijing','chengdu','hangzhou','chongqing','xian','suzhou','wuhan',
# #           'nanjing','tianjin','zhengzhou','changsha','dongguan','foshan','ningbo','qingdao','shenyang','hefei']
years = ['2011','2012','2013','2014','2015','2016','2017','2018']
# years = ['2018']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
# months = ['10','11','12']

# agent_list = [
#     "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1",
#     "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0) Gecko/20100101 Firefox/6.0",
#     "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
#     "Opera/9.80 (Windows NT 6.1; U; zh-cn) Presto/2.9.168 Version/11.50",
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36"
# ]


def getHtml(city, year, month, i):
    if i==0:
        url = 'http://lishi.tianqi.com/' + pinyin(city) + '/' + str(year) + str(month) + '.html'
    else:
        url = 'http://lishi.tianqi.com/' + pinyin(city) + str(i) + '/' + str(year) + str(month) + '.html'
#     url = 'http://lishi.tianqi.com/jiamusi' + '/' + str(year) + str(month) + '.html'
    print(url)
    
    try:
        request = urllib.request.Request(url)
        response = urllib.request.urlopen(request)
    except urllib.request.HTTPError as error:    # HTTP错误
        print('HTTPError')
        print('ErrorCode: %s' % error.code)
        time.sleep(10)
        if error.code==500:
            html,i =getHtml(city, year, month, i+1)
        else:
            html,i =getHtml(city, year, month, i)
        return html,i
    except urllib.request.URLError as error:     # URL错误
        print(error.reason)

#     user_agent = random.choice(agent_list) 
#     request = urllib.request.Request(url)
#     request.add_header('User-Agent', user_agent)
#     response = urllib.request.urlopen(request)
    html = response.read().decode('utf-8')
    pattern = re.compile('<title>(.*?)</title>')
    index=re.search(pattern,html)
#     print(index[0][7:7+len(city)])
    if index[0][7:7+len(city)]!=city:
        print(index[0][7:7+len(city)],city)
        html,i = getHtml(city, year, month, i+1)
    return html,i
 
def getTemp(html):
    text = "".join(html.split())
    rows = []
    patten = re.compile('<divclass="tian_three">(.*?)<divclass="lishidesc2">')
    table = re.findall(patten, text)
#     print('table:',table)
    patten1 = re.compile('<li(.*?)</li>')
    uls = re.findall(patten1, table[0])
#     print('uls:',uls)
    for ul in uls:
        time = re.findall('<divclass="th200">(.*?)</div>', ul)[0]
        patten2 = re.compile('<divclass="th140">(.*?)</div>')
        lis = re.findall(patten2, ul)
#         high = int(lis[0].replace('℃',''))
#         low = int(lis[1],replace('℃',''))

        high = lis[0].replace('℃','')
        high = int(high)
        low = lis[1].replace('℃','')
        low = int(low)
        weather = lis[2]
        wind = lis[3]
        rows.append({'time':time, 'high':high, 'low':low, 'weather':weather, 'wind':wind})
    return rows
 
 

if __name__ == "__main__":
    for fileName in filelist:
        if fileName.endswith('.json'):
            city = fileName.replace('.json','')
    #         city2=pinyin(city)
            cities.append(city)
    objPath='/Users/rumeng/Downloads/qingzang/trainSet/weather2/'
    cities=['兴宁']
    for city in cities:
        print(city)
        index=0
        for year in years:
            for month in months:
                html,index = getHtml(city, year, month,index)
#                 print(html)
#                 pattern = re.compile('<title>(.*?)</title>')
#                 index=re.search(pattern,html)
#                 print(index[0][7:9])
                rows = getTemp(html)
                for row in rows:
                    j=json.dumps(row,ensure_ascii=False)  
                    with open(objPath+city+'.json', 'a',encoding='utf-8-sig') as json_file:
                        json_file.write(j)
                        json_file.write('\n')
                time.sleep(1)
            time.sleep(2)
            
            



兴宁
http://lishi.tianqi.com/xingning/201101.html
http://lishi.tianqi.com/xingning/201102.html
http://lishi.tianqi.com/xingning/201103.html
http://lishi.tianqi.com/xingning/201104.html
http://lishi.tianqi.com/xingning/201105.html
http://lishi.tianqi.com/xingning/201106.html
http://lishi.tianqi.com/xingning/201107.html
http://lishi.tianqi.com/xingning/201108.html
http://lishi.tianqi.com/xingning/201109.html
http://lishi.tianqi.com/xingning/201110.html
http://lishi.tianqi.com/xingning/201111.html


KeyboardInterrupt: 

In [14]:
import requests
import json
import pandas as pd
from os import listdir

def get_data(address):
    url = f'https://restapi.amap.com/v3/place/text?s=rsv3&children=&key=6e79f6d236e295632f21b385e363b6e8&offset=1&page=1&extensions=all&city=110000&language=zh_cn&callback=jsonp_702014_&platform=JS&logversion=2.0&appname=https://lbs.amap.com/tools/picker&csid=6B8236BF-F308-412B-A131-37ECACA4657B&sdkversion=1.4.18&keywords={address}'
    headers = {
        'Cookie': 'isg=BF1da5qC4zjh7YcUZb1Z2uuwbj9XepHMUaXvRB8hi7Tj1n8I5ctXnFKAAFpQDamE; l=eBSZvekmLZNRY_v2BO5BFurza7793QO0zkPzaNbMiInca6_ltFa6pNCHwDueSdtf3tfX2exPrw3BrRh6894ZwBEhBJIqhySdVc9w-; tfstk=cd11Bd2yciCexl4OFIaUQpxpqdOVa6c68Pth5kpX0W-m-r_pJsbr43NgnZmpphTC.; cna=b9hLG9gLZwsCATsm1NJmFhvA',
        'Host': 'restapi.amap.com',
        'Referer': 'https://lbs.amap.com/',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15'
    }

    response = requests.get(url, headers=headers)
    result = response.text[14:-1]
    result = json.loads(result)  #如果报错，可以print(result)看一下
    if  result['status']=="0":  #运行过程中，可能出现无法根据地址搜索到省市区的情况，因此添加if和elif语句
        return 1
    elif not result['pois']:
        return 2
    else:    #city、province、space、district如果不知道怎么来的，可以print（result）看一下
        city = result['pois'][0]['cityname']
        province = result['pois'][0]['pname']
        space = result['pois'][0]['name']
        district = result['pois'][0]['adname']
#         return f'{space}省、市、区（县）是：{province}、{city}、{district}'
        return province,city,district

filelist2=listdir('/Users/rumeng/Downloads/qingzang/trainSet/company/')
cityDict={}

for fileName in filelist2:
    name = fileName.replace('.json','')
    province,city,district = get_data(name)
#     print(name,city)
    city = city.replace('市','')
    cityDict[name]=city
#     if city not in cityDict.keys():
#         cityDict[city]=[name]
#     else:
#         cityDict[city].append(name)
print(cityDict)




{'镇江': '镇江', '梅河口': '通化', '瑞安': '北京', '鹰潭': '鹰潭', '杭州': '杭州', '湘潭': '湘潭', '五家渠': '五家渠', '韩城': '渭南', '抚州': '抚州', '武威': '武威', '丽江': '丽江', '扬中': '镇江', '徐州': '徐州', '玉溪': '玉溪', '林芝': '林芝', '茂名': '茂名', '张家界': '张家界', '林州': '安阳', '潍坊': '潍坊', '西宁': '西宁', '嘉兴': '嘉兴', '淮北': '淮北', '项城': '周口', '保定': '保定', '池州': '池州', '乐陵': '德州', '通化': '通化', '湛江': '湛江', '许昌': '许昌', '格尔木': '海西蒙古族藏族自治州', '柳州': '柳州', '宣城': '宣城', '内江': '内江', '白银': '白银', '余姚': '宁波', '株洲': '株洲', '广州': '广州', '广安': '北京', '大庆': '大庆', '东莞': '东莞', '萍乡': '萍乡', '铜陵': '铜陵', '秦皇岛': '秦皇岛', '铜仁': '铜仁', '平顶山': '平顶山', '建德': '北京', '承德': '承德', '南阳': '南阳', '石嘴山': '石嘴山', '丹阳': '镇江', '桂林': '桂林', '沧州': '沧州', '台山': '江门', '泸州': '泸州', '平湖': '嘉兴', '九江': '九江', '.DS_Store': '北京', '石河子': '石河子', '胶州': '青岛', '溧阳': '常州', '新余': '新余', '晋江': '泉州', '北海': '北京', '抚顺': '抚顺', '靖江': '泰州', '眉山': '眉山', '乐清': '温州', '清远': '清远', '阜阳': '阜阳', '包头': '包头', '玉环': '台州', '营口': '营口', '景德镇': '景德镇', '偃师': '洛阳', '龙岩': '龙岩', '重庆': '重庆', '义马': '三门峡', '三亚': '三亚', '哈尔滨': '哈尔滨', '兴化': '北京', '淄博':

In [2]:
cityDict = {'镇江': '镇江', '梅河口': '通化', '瑞安': '北京', '鹰潭': '鹰潭', '杭州': '杭州', '湘潭': '湘潭', '五家渠': '五家渠', '韩城': '渭南', '抚州': '抚州', '武威': '武威', '丽江': '丽江', '扬中': '镇江', '徐州': '徐州', '玉溪': '玉溪', '林芝': '林芝', '茂名': '茂名', '张家界': '张家界', '林州': '安阳', '潍坊': '潍坊', '西宁': '西宁', '嘉兴': '嘉兴', '淮北': '淮北', '项城': '周口', '保定': '保定', '池州': '池州', '乐陵': '德州', '通化': '通化', '湛江': '湛江', '许昌': '许昌', '格尔木': '海西蒙古族藏族自治州', '柳州': '柳州', '宣城': '宣城', '内江': '内江', '白银': '白银', '余姚': '宁波', '株洲': '株洲', '广州': '广州', '广安': '北京', '大庆': '大庆', '东莞': '东莞', '萍乡': '萍乡', '铜陵': '铜陵', '秦皇岛': '秦皇岛', '铜仁': '铜仁', '平顶山': '平顶山', '建德': '北京', '承德': '承德', '南阳': '南阳', '石嘴山': '石嘴山', '丹阳': '镇江', '桂林': '桂林', '沧州': '沧州', '台山': '江门', '泸州': '泸州', '平湖': '嘉兴', '九江': '九江', '.DS_Store': '北京', '石河子': '石河子', '胶州': '青岛', '溧阳': '常州', '新余': '新余', '晋江': '泉州', '北海': '北京', '抚顺': '抚顺', '靖江': '泰州', '眉山': '眉山', '乐清': '温州', '清远': '清远', '阜阳': '阜阳', '包头': '包头', '玉环': '台州', '营口': '营口', '景德镇': '景德镇', '偃师': '洛阳', '龙岩': '龙岩', '重庆': '重庆', '义马': '三门峡', '三亚': '三亚', '哈尔滨': '哈尔滨', '兴化': '北京', '淄博': '淄博', '慈溪': '宁波', '洛阳': '洛阳', '达州': '达州', '普宁': '揭阳', '吉首': '湘西土家族苗族自治州', '敦化': '延边朝鲜族自治州', '枝江': '宜昌', '永康': '北京', '廊坊': '廊坊', '福泉': '北京', '邯郸': '邯郸', '晋中': '晋中', '常德': '常德', '临汾': '临汾', '荆门': '荆门', '黄石': '黄石', '曲阜': '济宁', '白山': '白山', '丽水': '北京', '如皋': '南通', '菏泽': '菏泽', '常熟': '苏州', '枣庄': '枣庄', '陇南': '陇南', '兰州': '兰州', '温州': '温州', '磐石': '吉林', '丹东': '丹东', '东阳': '北京', '龙口': '烟台', '湖州': '湖州', '张家港': '苏州', '长治': '长治', '常州': '常州', '马鞍山': '北京', '蚌埠': '蚌埠', '晋城': '晋城', '绵阳': '绵阳', '石家庄': '石家庄', '银川': '银川', '大理': '大理白族自治州', '南宁': '南宁', '衢州': '衢州', '滨州': '滨州', '莱芜': '济南', '鞍山': '鞍山', '临沂': '临沂', '南平': '北京', '鄂尔多斯': '北京', '安顺': '北京', '温岭': '台州', '德州': '德州', '滕州': '枣庄', '句容': '镇江', '无锡': '无锡', '铁岭': '铁岭', '莱州': '烟台', '运城': '运城', '随州': '随州', '南通': '南通', '襄阳': '襄阳', '启东': '南通', '攀枝花': '攀枝花', '安庆': '安庆', '六安': '六安', '汉川': '孝感', '四会': '肇庆', '河源': '河源', '邵阳': '邵阳', '武汉': '武汉', '长沙': '长沙', '濮阳': '濮阳', '大连': '大连', '天水': '天水', '周口': '周口', '淮安': '淮安', '亳州': '亳州', '梅州': '梅州', '牡丹江': '牡丹江', '神木': '北京', '高碑店': '北京', '梧州': '梧州', '上饶': '上饶', '沈阳': '沈阳', '崇州': '成都', '阳春': '北京', '乐山': '乐山', '西安': '西安', '盘锦': '盘锦', '南昌': '南昌', '宜兴': '无锡', '辽阳': '辽阳', '寿光': '潍坊', '义乌': '金华', '中山': '北京', '宜宾': '北京', '黄山': '黄山', '阜康': '北京', '惠州': '惠州', '太仓': '北京', '连云港': '连云港', '长葛': '许昌', '喀什': '喀什地区', '贺州': '贺州', '高邮': '扬州', '鹤壁': '鹤壁', '福州': '福州', '新沂': '徐州', '巢湖': '合肥', '本溪': '本溪', '北京': '北京', '开封': '开封', '潜山': '安庆', '德阳': '德阳', '辽源': '辽源', '海东': '北京', '莱阳': '烟台', '济宁': '济宁', '锦州': '锦州', '海宁': '嘉兴', '郑州': '郑州', '中卫': '中卫', '咸阳': '咸阳', '珠海': '珠海', '宁德': '宁德', '怀化': '怀化', '乌鲁木齐': '乌鲁木齐', '遵义': '遵义', '潜江': '潜江', '曲靖': '曲靖', '岳阳': '岳阳', '赤峰': '赤峰', '呼和浩特': '呼和浩特', '葫芦岛': '葫芦岛', '忻州': '忻州', '宜春': '宜春', '江阴': '无锡', '自贡': '自贡', '聊城': '聊城', '宁波': '宁波', '拉萨': '拉萨', '赣州': '赣州', '新乡': '新乡', '云浮': '云浮', '阿克苏': '阿克苏地区', '阜新': '阜新', '遂宁': '遂宁', '天津': '天津', '宁乡': '长沙', '江门': '江门', '南京': '南京', '长春': '长春', '昆山': '苏州', '什邡': '德阳', '驻马店': '驻马店', '商丘': '商丘', '塔城': '塔城地区', '莆田': '北京', '安阳': '安阳', '克拉玛依': '克拉玛依', '吉林': '吉林', '衡水': '衡水', '济南': '济南', '汕头': '汕头', '临海': '台州', '太原': '太原', '阳泉': '阳泉', '海口': '海口', '台州': '台州', '荆州': '荆州', '嘉峪关': '嘉峪关', '库尔勒': '巴音郭楞蒙古自治州', '福清': '福州', '宝鸡': '宝鸡', '青岛': '青岛', '威海': '威海', '阿拉尔': '阿拉尔', '齐齐哈尔': '齐齐哈尔', '六盘水': '六盘水', '昆明': '昆明', '贵阳': '贵阳', '沅江': '益阳', '大同': '大同', '泰兴': '泰州', '上海': '上海', '益阳': '益阳', '深圳': '深圳', '南安': '北京', '揭阳': '揭阳', '信阳': '信阳', '乐平': '北京', '滁州': '滁州', '西昌': '凉山彝族自治州', '明光': '北京', '绍兴': '绍兴', '唐山': '唐山', '合肥': '合肥', '厦门': '厦门', '苏州': '苏州', '荣成': '威海', '吉安': '吉安', '江山': '北京', '焦作': '焦作', '金华': '金华', '乌海': '乌海', '东营': '北京', '淮南': '淮南', '昌吉': '昌吉回族自治州', '河池': '河池', '三明': '三明', '韶关': '韶关', '酒泉': '酒泉', '泰安': '泰安', '武穴': '黄冈', '泉州': '泉州', '咸宁': '咸宁', '桐乡': '嘉兴', '泰州': '泰州', '衡阳': '衡阳', '漳州': '漳州', '鄂州': '鄂州', '鹤山': '江门', '诸城': '潍坊', '峨眉山': '乐山', '昌邑': '潍坊', '成都': '成都', '日照': '日照', '郴州': '郴州', '嵊州': '绍兴', '招远': '烟台', '芜湖': '芜湖', '宁国': '宣城', '扬州': '扬州', '即墨': '青岛', '当阳': '宜昌', '济源': '济源', '邢台': '邢台', '盐城': '盐城', '佛山': '佛山', '漯河': '漯河', '永城': '商丘', '七台河': '七台河', '诸暨': '绍兴', '十堰': '十堰', '烟台': '烟台', '兴宁': '梅州', '潮州': '潮州', '宿迁': '宿迁', '肇庆': '肇庆', '佳木斯': '佳木斯', '舟山': '舟山', '宜昌': '宜昌'}

In [10]:
sourcePath = '/Users/rumeng/Downloads/qingzang/trainSet/company/'
objPath = '/Users/rumeng/Downloads/qingzang/trainSet2/company/'
filelist1 = listdir(sourcePath)
for fileName in filelist1:
    print(fileName)
    if '.json' in fileName:
        objCity = cityDict[fileName.replace('.json','')]
        with open(objPath+objCity+'.json', 'a',encoding='utf-8-sig') as json_file:
            with open(sourcePath+fileName, "r",encoding='utf-8-sig') as f:
                for line in f.readlines():
#                     print(line)
                    j=json.loads(line)
                    j=json.dumps(j,ensure_ascii=False)   
                    json_file.write(j)
                    json_file.write('\n')

镇江.json
梅河口.json
瑞安.json
鹰潭.json
杭州.json
湘潭.json
五家渠.json
韩城.json
抚州.json
武威.json
丽江.json
扬中.json
徐州.json
玉溪.json
林芝.json
茂名.json
张家界.json
林州.json
潍坊.json
西宁.json
嘉兴.json
淮北.json
项城.json
保定.json
池州.json
乐陵.json
通化.json
湛江.json
许昌.json
格尔木.json
柳州.json
宣城.json
内江.json
白银.json
余姚.json
株洲.json
广州.json
广安.json
大庆.json
东莞.json
萍乡.json
铜陵.json
秦皇岛.json
铜仁.json
平顶山.json
建德.json
承德.json
南阳.json
石嘴山.json
丹阳.json
桂林.json
沧州.json
台山.json
泸州.json
平湖.json
九江.json
.DS_Store
石河子.json
胶州.json
溧阳.json
新余.json
晋江.json
北海.json
抚顺.json
靖江.json
眉山.json
乐清.json
清远.json
阜阳.json
包头.json
玉环.json
营口.json
景德镇.json
偃师.json
龙岩.json
重庆.json
义马.json
三亚.json
哈尔滨.json
兴化.json
淄博.json
慈溪.json
洛阳.json
达州.json
普宁.json
吉首.json
敦化.json
枝江.json
永康.json
廊坊.json
福泉.json
邯郸.json
晋中.json
常德.json
临汾.json
荆门.json
黄石.json
曲阜.json
白山.json
丽水.json
如皋.json
菏泽.json
常熟.json
枣庄.json
陇南.json
兰州.json
温州.json
磐石.json
丹东.json
东阳.json
龙口.json
湖州.json
张家港.json
长治.json
常州.json
马鞍山.json
蚌埠.json
晋城.json
绵阳.json
石家庄.json
银川.json
大理.json
南宁.json
衢

In [11]:
filelist1=listdir('/Users/rumeng/Downloads/qingzang/trainSet2/company/')
filelist2=listdir('/Users/rumeng/Downloads/qingzang/trainSet2/weather/')
notAddList = []
for fileName in filelist1:
    if fileName.replace('.json','')+'Temp.json' not in filelist2:
        notAddList.append(fileName)
print(notAddList)

['.DS_Store', 'noWeather', '巴州.json', '舟山.json']


In [19]:
url = 'https://api.map.baidu.com/weather/v1/?district_id=110100&data_type=all&ak=z29V2EL1hlYaD0XOXOp1xmq3DM9sjtCW'
try:
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
except urllib.request.HTTPError as error:    # HTTP错误
    print('HTTPError')
    print('ErrorCode: %s' % error.code)
except urllib.request.URLError as error:     # URL错误
    print(error.reason)
html = response.read().decode('utf-8')
print(html)
html=eval(html)
for i in html.keys():
    print(i)

myDict={}
myDict['now']={
    'text':html['result']['now']['text'],
    'temp':html['result']['now']['temp'],
    'wind_class':html['result']['now']['wind_class'],
    'wind_dir':html['result']['now']['wind_dir']
}
myDict['day']={}
for i in range(3):
    dayInfo=html['result']['forecasts'][i]
    myDict['day'][dayInfo['date']]={
        'text':dayInfo['text_day'],
        'high':dayInfo['high'],
        'low':dayInfo['low'],
        'wind_class':dayInfo['wc_day'],
        'wind_dir':dayInfo['wd_day'],
        'week':dayInfo['week']
    }
print(myDict)


{"status":0,"result":{"location":{"country":"中国","province":"北京市","city":"北京市","name":"北京","id":"110100"},"now":{"text":"晴","temp":28,"feels_like":29,"rh":50,"wind_class":"2级","wind_dir":"东北风","uptime":"20220915141000"},"forecasts":[{"text_day":"多云","text_night":"多云","high":30,"low":18,"wc_day":"<3级","wd_day":"东南风","wc_night":"<3级","wd_night":"东南风","date":"2022-09-15","week":"星期四"},{"text_day":"多云","text_night":"多云","high":33,"low":20,"wc_day":"<3级","wd_day":"西南风","wc_night":"<3级","wd_night":"西南风","date":"2022-09-16","week":"星期五"},{"text_day":"多云","text_night":"多云","high":32,"low":20,"wc_day":"<3级","wd_day":"西南风","wc_night":"<3级","wd_night":"北风","date":"2022-09-17","week":"星期六"},{"text_day":"晴","text_night":"晴","high":31,"low":17,"wc_day":"<3级","wd_day":"北风","wc_night":"<3级","wd_night":"北风","date":"2022-09-18","week":"星期日"},{"text_day":"多云","text_night":"多云","high":28,"low":15,"wc_day":"<3级","wd_day":"北风","wc_night":"<3级","wd_night":"东南风","date":"2022-09-19","week":"星期一"}]},"message":"

In [3]:
data = pd.read_csv("/Users/rumeng/Downloads/qingzang/trainSet2/weather_district_id.csv")
print(data)
above5 = ['镇江', '杭州', '徐州', '潍坊', '西宁', '嘉兴', '保定', '许昌', '柳州', '株洲', 
          '广州', '东莞', '铜陵', '南阳', '桂林', '包头', '龙岩', '重庆', '哈尔滨', '淄博', 
          '洛阳', '荆门', '兰州', '温州', '湖州', '常州', '绵阳', '石家庄', '银川', '南宁', 
          '衢州', '滨州', '鞍山', '德州', '无锡', '南通', '襄阳', '武汉', '长沙', '大连', 
          '梅州', '沈阳', '乐山', '西安', '南昌', '惠州', '连云港', '福州', '北京', '德阳', 
          '济宁', '郑州', '珠海', '乌鲁木齐', '岳阳', '呼和浩特', '宁波', '拉萨', '赣州', '新乡',
          '天津', '江门', '南京', '长春', '吉林', '济南', '汕头', '太原', '海口', '台州',
          '宝鸡', '青岛', '威海', '昆明', '贵阳', '上海', '益阳', '深圳', '揭阳', '滁州', 
          '绍兴', '唐山', '合肥', '厦门', '苏州', '焦作', '金华', '泉州', '泰州', '衡阳', 
          '漳州', '成都', '芜湖', '扬州', '盐城', '佛山', '烟台', '潮州', '宿迁', '肇庆', 
          '宜昌']
myDict={}
for i in range(len(data)):
    districtcode=data['districtcode'].values[i]
    district=data['district'].values[i]
    if district in above5:
        myDict[district]=districtcode
print(len(myDict))
print(myDict)

       areacode  districtcode  city_geocode city  district_geocode district  \
0     101010100        110100        110100  北京市            110100       北京   
1     101011600        110101        110100  北京市            110101       东城   
2     101011700        110102        110100  北京市            110102       西城   
3     101010300        110105        110100  北京市            110105       朝阳   
4     101010900        110106        110100  北京市            110106       丰台   
...         ...           ...           ...  ...               ...      ...   
3395  101341601        719010        719010  嘉义县            719010      嘉义县   
3396  101341701        719011        719011  屏东县            719011      屏东县   
3397  101341801        719012        719012  台东县            719012      台东县   
3398  101341901        719013        719013  花莲县            719013      花莲县   
3399  101342001        719014        719014  澎湖县            719014      澎湖县   

             lon        lat sta_fc sta_rt province 